In [1]:
import json

In [2]:
f = open('mpqa_csds.json', 'r', encoding='utf-8')

In [3]:
data = json.load(f)

In [4]:
import re

def replace_punc(clean_text, offset, offset_list):
    poss_diff = 0
    dash_diff = 0
    s = clean_text
    s, off1 = re.subn(' \.', '.', s)
    s, off2 = re.subn(' ,', ',', s)
    s, off3 = re.subn(' ;', ';', s)
    s, off4 = re.subn(' :', ':', s)
    s, off5 = re.subn(' POSS', '\'s', s)
    s, off6 = re.subn(' n\'t', 'n\'t', s)
    s, off7 = re.subn(' %', '%', s)
    s, off8 = re.subn('$ ', '$', s)
    s, off8 = re.subn('  --  ', ' -- ', s)
    s, off9 = re.subn('\( ', '(', s)
    s, off10 = re.subn(' \)', ')', s)
    s, off11 = re.subn(' !', '!', s)
    s, off12 = re.subn(' ?', '?', s)
    if off8 == 1:
        dash_diff = 2
    if off5 == 1:
        poss_diff = 3
    adjust_val = off1 + off2 + off3 + off4 + poss_diff + off6 + off7 + dash_diff + off9 + off10 + off11 + off12
    offset -= adjust_val
    if (off1 + off2 + off3 + off4 + poss_diff + off6 + off7 + dash_diff + off9 + off10 + off11 + off12 > 0):
        offset_list[-1] -= adjust_val
    return s, offset, offset_list

In [5]:
def quote_helper(offset, offset_list, quote_bool, index, clean_text, w_text):
    if not quote_bool:
        #when only one quote is present and it's the last one in the sentence
        if index == len(w_text)-1:
            clean_text += w_text[index]
            offset_list.append(offset)
            offset += 1
            return offset, offset_list, quote_bool, index, clean_text, w_text
        clean_text += ' ' + w_text[index] + w_text[index+1]
        offset += 1
        offset_list.append(offset)
        offset += len(w_text[index])
        offset_list.append(offset)

        #for the word itself
        index+=1
        offset += len(w_text[index])
        quote_bool = True
    else:
        clean_text += w_text[index]
        offset_list.append(offset)
        offset += 1
        quote_bool = False
    return offset, offset_list, quote_bool, index, clean_text, w_text

In [6]:
#returns clean text and offset_list that contains the start offset of every token in the clean text, given a list of tokens that comprise the clean text
# w_head is needed just to check if the head is only one single quote, which sometimes occurs
def assemble_tokens(w_text, w_head):
    clean_text = ''
    offset = 0
    offset_list = [0]
    index = 0
    in_double_quote = False
    in_single_quote = False
    while index < len(w_text):
        if (index == 0):
            #if first index is quote:
            if w_text[index] == '"':
                if len(w_head) == 1 and w_head[0] == '"':
                    clean_text += w_text[index]
                    offset += len(w_text[index])
                    offset_list.append(offset)
                    return clean_text, offset_list
                clean_text += w_text[index] + w_text[index+1]
                offset += len(w_text[index])
                offset_list.append(offset)

                #for the word itself
                index+=1
                offset += len(w_text[index])
                in_double_quote = True
            else:
                clean_text += w_text[index]
                offset += len(w_text[index])
        elif w_text[index] == '"':
            offset, offset_list, in_double_quote, index, clean_text, w_text = quote_helper(offset, offset_list, in_double_quote, index, clean_text, w_text)
        elif w_text[index] == '\'':
            offset, offset_list, in_single_quote, index, clean_text, w_text = quote_helper(offset, offset_list, in_single_quote, index, clean_text, w_text)
        else:
            clean_text += ' '
            offset += 1
            offset_list.append(offset)
            clean_text += w_text[index]    
            offset += len(w_text[index])
        clean_text, offset, offset_list = replace_punc(clean_text, offset, offset_list)
        index += 1
    return clean_text, offset_list

In [7]:
#returns clean head of csds object
def return_clean_head(csds_obj):
    punc_list = ['.', ',', ';', ':', '\"', '\'', '!', '?']
    s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end', 'w_head'])
    if s1.issubset(obj.keys()):
        print('ORIGINAL:', obj['clean_head'])
        w_text = obj['w_text']
        w_head = obj['w_head']
        for ind in range(len(w_text)):
            if w_text[ind] == '\'s':
                w_text[ind] = "POSS"
        clean, offset_list = assemble_tokens(w_text, w_head)
        print(offset_list)
        w_start, w_end = obj['w_head_span'] 
        #if head is whole sentence
        if w_start == w_end:
            return ''
        s_head = offset_list[w_start]
        #if rest of sentence is part of head, prevents list index out of range error
        if w_end == len(offset_list):
            return clean[s_head:]
        else:
            e_head = offset_list[w_end]
            
        #deal with punctuation since they won't have spaces before them (commas will be two spaces away, quotes one space away)
        if offset_list[-1] == e_head or clean[offset_list[w_end+1]-2] in punc_list or clean[offset_list[w_end+1]-1] in punc_list:
            #if punctuation is last character in the sentence, you want to include it
            if w_head[-1] in punc_list and w_text[-1] == w_head[-1]:
                return clean[s_head:]
            else:
                return clean[s_head:e_head]
        else:
            return clean[s_head:e_head-1]

In [8]:
# just to test if functions work
def testing(l_bound, limit):
    count = 0
    punc_list = ['.', ',', ';', ':', '\"', '\'', '!', '?']
    for obj in data:
        if count > limit:
            break
        if count < l_bound:
            count += 1
            continue
        s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end', 'w_head'])
        w_text = obj['w_text']
        w_head = obj['w_head']
        for ind in range(len(obj['w_text'])):
            if w_text[ind] == '\'s':
                w_text[ind] = "POSS"
        if s1.issubset(obj.keys()):
            clean, offset_list = assemble_tokens(w_text, w_head)
            w_start, w_end = obj['w_head_span'] 
            print('-------------------------------------------------------')
            #print(clean)
            #print(offset_list)
            # print(obj['w_text'])
            print('ORIGINAL:', obj['clean_head'])
            #print("head span", obj['w_head_span'])
            
            #head is whole sentence
            if w_start == w_end:
                print('     NEW:', '')
                count += 1
                print(count)
                continue
            s_head = offset_list[w_start]
            if w_end == len(offset_list):
                print(' b   NEW:', clean[s_head:])
                count += 1
                print(count)
                continue
            else:
                e_head = offset_list[w_end]
            
            #deal with punctuation since they won't have spaces before them (commas will be two spaces away, quotes one space away)
            if offset_list[-1] == e_head or clean[offset_list[w_end+1]-2] in punc_list or clean[offset_list[w_end+1]-1] in punc_list:
                #if whole punctuation is last character in the sentence, you want to include it
                if w_head[-1] in punc_list and w_text[-1] == w_head[-1]:
                    print(' a   NEW:', clean[s_head:])
                else:
                    print(' a   NEW:', clean[s_head:e_head])
            else:
                print('     NEW:', clean[s_head:e_head-1])
        count+=1
        print(count)

In [ ]:
testing(40000, 41000)
print('DONEEEEEEE')